# Maximum Likelihood: Examples in R
#### REFERENCE: Maximum Likelihood Programming in R
Marco R. Steenbergen
Department of Political Science
University of North Carolina, Chapel Hill

## MLE in R entails a two-step process. 
## *1-  declare the log-likelihood function, which is done in general terms. 
## *2-   optimize the log-likelihood function, which is done in terms of a particular data set. 
The log-likelihood function and optimization command may be typed interactively
into the R command window or they may be contained in a text file. 
In general it is advisable to save functions into a text file, especially if you plan
on using them frequently.

## EXAMPLE: Poisson Log Likelihood

## The log log likelihood is:
$$ L(\mu)=\sum_i y_i ln(\mu)- n \mu - \sum_i ln(y_i !)$$

## Since the last term does not include the parameter, $\mu$, it can be safely ignored. Hence:
$$  L(\mu)\simeq\sum_i y_i ln(\mu)- n \mu $$
We can program this function using the following syntax:

In [94]:
options(repr.plot.width = 8, repr.plot.height = 8)


In [45]:
poisson.lik<-function(y,par){
n<-length(y)
mu=par
logl<-sum(y)*log(mu)-n*mu
return(-logl)
}

Here poisson.lik is the name of the log-likelihood function; this name will be
used in the **optim** command. 

The unknown of parameter is  $\mu$ and needs to be estimated.

Further, y is the placeholder for the data.

Since the log-likelihood function
requires knowledge of the sample size, we obtain this using **n<-nrow(y)**.

The expression for logl contains the kernel of the log-likelihood function. 
Finally, we ask R to return -1 times the log-likelihood function. 

Why? Because the function **optim** minimizes!


# Optimizing the Log-Likelihood
## Once the log-likelihood function has been declared, then the optim command
## can be invoked. 
## The minimal specification of this command is

## **optim(starting values, log-likelihood, data)**

**starting values** is a vector of starting values, 

**log-likelihood** is the name of the log-likelihood function that you seek to maximize, and 
**data** declares the data for the estimation. 

This specification causes R to use the Nelder-Mead algorithm. If you want to use the BFGS algorithm you should include
the method="BFGS" option. For the L-BFGS-B algorithm you should declare
method="L-BFGS-B". (see **?optim** for details)

Imagine that we have a vector data that consists of draws from
    a Poisson distribution with unknown $\lambda$. We seek to estimate this parameter and
have already declared the log-likelihood function as poisson.lik. Estimation
using the BFGS algorithm now commences as follows


In [44]:
data=rpois(n=10,lambda = 5)
data
length(data)

[1] 7 7 2 5 2 5 6 6 8 6

[1] 10

In [46]:
poisson.lik

function(y,par){
n<-length(y)
mu=par
logl<-sum(y)*log(mu)-n*mu
return(-logl)
}

In [52]:
resPois=optim(par=1,poisson.lik,y=data,method="BFGS")


Here 1 is the starting value for the algorithm. Since the log-likelihood function
refers to generic data objects as y, it is important that the vector data is equated
with y.


In [48]:
mean(data)

[1] 5.4

In [53]:
cbind('direct'=c('mean'=mean(data)),
'optim'=resPois$par)

,direct,optim
mean,5.4,5.400001


## Normal Log Likelihood


### Imagine that we have a sample that was drawn from a normal
### distribution with unknown mean, $\mu$, and variance, $\sigma^2$. The objective is to
### estimate these parameters. The normal log-likelihood function is given by:

$$
0.5n*ln(2\pi)- 0.5*n*ln(\sigma^2)-\frac{1}{2*\sigma^2}\sum(y_i-\mu)^2
$$

### We can define the function:


In [32]:

llik = function(x,par){
m=par[1]
s=par[2]
n=length(x)
# log of the normal likelihood
# -n/2 * log(2*pi*s^2) + (-1/(2*s^2)) * sum((x-m)^2)
ll = -(n/2)*(log(2*pi*s^2)) + (-1/(2*s^2)) * sum((x-m)^2)
# return the negative to maximize rather than minimize
return(-ll)
    }

Here **par** is a vector containing the two parameters of interest. 

We declare the elements of this vector in the first two lines of the bracketed part of the program.

Specifically:

-the first element (par[1]) is equal to $\mu$, 

-the second element (par[2]) is equal to $\sigma^2$. 

The remainder of the program sets:

-the sample size,

-specifies the log-likelihood function, 

-return the negative of this function.

In [95]:
plot(seq(-3,3,.1),-1*sapply(seq(-3,3,.1),FUN=llik,par=c(0,1)),type='l',
ylab='',xlab='')

In [96]:
# negative log likelihood curve
# just to see what the funciton produces we can plot it.
plot(seq(-3,3,.1),sapply(seq(-3,3,.1),FUN=llik,par=c(0,1)),type='l',
ylab='',xlab='')

## Generate the data

In [50]:
set.seed(1123) # for consistency set the seed explicitly.
#first simulate some normal data with expected mean of 0 and sd of 1
x = rnorm(100)
# scale the data the way that we would like
x = x/sd(x) * 8 # sd of 8
x = x-mean(x) + 10 # mean of 10
c('mean'=mean(x),'sd'=sd(x)) # double check

mean   sd 
  10    8

In [97]:
# histogram 
hist(x, freq=FALSE,col='tan')
lines(density(x),col='red',lwd=2)

In [98]:
# specify the single value normal probability function
norm_lik = function(x, m, s){
y = 1/sqrt(2*pi*s^2)*exp((-1/(2*s^2))*(x-m)^2)
}
# and plot it just to make sure
plot(seq(-3,3,.1),sapply(seq(-3,3,.1),FUN=norm_lik,m=0,s=1),type='l',
ylab='',xlab='', main='Gaussian Normal')

In [38]:
res0 = optim(par=c(.5,.5), llik, x=x)

In [41]:
res0

$par
[1] 10.001693  7.975965

$value
[1] 349.3359

$counts
function gradient 
      95       NA 

$convergence
[1] 0

$message
NULL

In [42]:
cbind('direct'=c('mean'=mean(x),'sd'=sd(x)),
'optim'=res0$par)

,direct,optim
mean,10,10.001693
sd,8,7.975965


# **ASSIGNMENT**: 

## FOLLOWING THE OUTLINE PROVIDED FOR THE NORMAL AND POISSON PARAMETERS:

## * IMPLEMENT THE MAXIMUM LIKELIHOOD ESTIMATION OF THE EXPONENTIAL DISTRIBUTION PARAMETER, USING THE "OPTIM" R FUNTION.

## * COMMENT YOUR WORK

## * COMPARE THE DIRECT ESTIMATION WITH THE OPTIMAL VALUE YOU FIND